# Analyzing borrowers’ risk of defaulting

This project generates a report for a bank’s loan division. The goal of the report is to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

This report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

## Data Overview

In [1]:
# import libraries
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
credit_data = pd.read_csv('/datasets/credit_scoring_eng.csv') #read file

display(credit_data.head(10)) #print the first 10 row of the data for general information

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [3]:
credit_data.info() # print the info of the data, length of data, data types, missing values etc'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
credit_data.describe() # learn mote about important values in the data like mean, median, min and max..

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [5]:
credit_data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

From the info table we can learn that there are 2,174 missing values in the "days_employed" and "total_income" columns, which is ~10% of the data. 
Also, the "days_employed" columns has weird numbers like (-) days which doesn't make sense and might be a typing error and should be changed to absolute values and the maximum days 401755 which translates to 1,100 years of employment which doesn't make any sense either. Since this column is not part of our investigation we can just ignore it. 
As for the missing values in the "total_income" column we will need to replace them.
All the data types seem to be the appropriate ones, we can change the "total_income" from float64 to int64 just to make it easier to perform any math calculations on this column. 
Looking at the describe table, "Children" column has (-) value and max value of 20 which is pretty unusual, we should change it to something more reasonable like 2 (assuming it's a typo), will be handled in the next step. 
In the "dob_year" which is suppose to be the age we have a minimum values of 0 which is probably a typo, since this is not something we analyze in this project we can leave it as is. 



## Data preprocessing

### Processing missing values

Before moving to missing values I will fix the unreasonable values we saw in 'children' column.

In [6]:
#taking care of the unreasonable values 20, -1
credit_data['children'] = credit_data['children'].replace(-1, 1)
credit_data['children'] = credit_data['children'].replace(20, 2)

In [7]:
print(credit_data.isnull().sum()) #looking for missing values


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


There are 2174 missing values in the table in the "days_employed" and "total_income" columns. The missing values type is float64. 

In [8]:
credit_data[credit_data.days_employed.isnull()] #looking for a common ground in the missing "days_employed" rows

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


No common ground was observed in the missing rows, different education, ages, family status, income type..

In [9]:
#checking is the missing rows for "days_employed" are the same missing rows for "total_income"
credit_data[credit_data.days_employed.notna() & credit_data.total_income.isnull()] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


All the missing values in "days_employed" are of the same people with no "total_income" data.
It could be that the reason for these missing values is that these people currently don't have an income and and they are not employed, even though they do have an income type. Or maybe there was a problem in the form in these sections.

In [10]:
#fill in the missing data of "total_income" with the median total income according to the income type and education level
credit_data['total_income'] = credit_data['total_income'].fillna(credit_data.groupby(['income_type', 'education_id'])['total_income'].transform('median'))

In [11]:
credit_data = credit_data.drop('days_employed',1) #remove "days_employed" from the dataset

In [12]:
credit_data.isnull().sum() #checking the fill in from previous step

children            0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

There are no more missing values in the "total_income" column and the "days_employed" column was deleted, as expected.

Approximately 10% of the data rows are missing the 'days_employed' and 'total_income' information. 
Since we are not using the "days_employed" data for our analysis I removed this column using .drop() method and replace missing values only for the "total_income" column which is required for the analysis. 
I decided to replace the missing values using the .fillna() method, with the median value of a population group with the same income type and same education which are two factors that can affect someone's income. 

### Data type replacement

In [13]:
credit_data['total_income'] = credit_data['total_income'].astype(int) #changing data type of 'total_income' 

Only for the purposes of faster math calculations I changed the data type of "total_income" from float to int using .astype() method. 
All other columns are represented with the appropriate type. 

### Processing duplicates

In [14]:
len(credit_data[credit_data.duplicated()]) # checking for duplicated data using .duplicated() method

54

In [15]:
credit_data[credit_data.duplicated()]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,secondary education,1,married,0,F,employee,0,21848,purchase of the house for my family
4182,1,34,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,26502,wedding ceremony
4851,0,60,secondary education,1,civil partnership,1,F,retiree,0,18374,wedding ceremony
5557,0,58,secondary education,1,civil partnership,1,F,retiree,0,18374,to have a wedding
7808,0,57,secondary education,1,civil partnership,1,F,retiree,0,18374,having a wedding
8583,0,58,bachelor's degree,0,unmarried,4,F,retiree,0,23078,supplementary education
9238,2,34,secondary education,1,married,0,F,employee,0,21848,buying property for renting out
9528,0,66,secondary education,1,widow / widower,2,F,retiree,0,18374,transactions with my real estate
9627,0,56,secondary education,1,married,0,F,retiree,0,18374,transactions with my real estate
10462,0,62,secondary education,1,married,0,F,retiree,0,18374,buy commercial real estate


We have 54 duplicates, which is only 2% of the data, but we can see that some of the 'education' values are the same with lower and upper cases. we will change them to lower case and see what happens.

In [16]:
credit_data['education'] = credit_data['education'].str.lower() #changing 'education' values to lowercase
len(credit_data[credit_data.duplicated()]) # counting duplicates

71

In [17]:
credit_data[credit_data.duplicated()]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,secondary education,1,married,0,F,employee,0,21848,purchase of the house for my family
3290,0,58,secondary education,1,civil partnership,1,F,retiree,0,18374,to have a wedding
4182,1,34,bachelor's degree,0,civil partnership,1,F,employee,0,26502,wedding ceremony
4851,0,60,secondary education,1,civil partnership,1,F,retiree,0,18374,wedding ceremony
5557,0,58,secondary education,1,civil partnership,1,F,retiree,0,18374,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...
20702,0,64,secondary education,1,married,0,F,retiree,0,18374,supplementary education
21032,0,60,secondary education,1,married,0,F,retiree,0,18374,to become educated
21132,0,47,secondary education,1,married,0,F,employee,0,21848,housing renovation
21281,1,30,bachelor's degree,0,married,0,F,employee,0,26502,buy commercial real estate


We can see that even after changing the 'education' to lower case we still get a small portion of that data as duplicates. We can remove these rows using drop_duplicates() method.

In [18]:
credit_data = credit_data.drop_duplicates().reset_index(drop = True) # dropping all the duplicated rows
len(credit_data[credit_data.duplicated()]) # making sure we deleted all duplicates

0

There are 54 duplicated rows in the data, this could mean that we might have duplications of some forms. 
We can see that the 'education column has duplicates but with lower and upper case letter (which might imply that the forms were filled by hand by different people) so we changed it to be all lower case and we received a 71 duplicates which is a similar portion of the dataset.
Although we can have duplicates in the data from different clients since there's no specific user ID, diffetnt people can have the same income type, total income, family status, number of children, or education I still decided to remove this data since we can't be sure if it's a duplication of the same client and it's a small portion of the data so I assume it will not effect on the analysis by much. 

### Categorizing Data

In [19]:
# categorizing the data by income levels
credit_data['total_income'].describe() #learning more about the distribution of income in our dataset

count     21454.000000
mean      26472.049128
std       15726.191978
min        3306.000000
25%       17202.000000
50%       22993.000000
75%       31728.750000
max      362496.000000
Name: total_income, dtype: float64

We can categorize the data accoring to the levels in the describe table

In [20]:
#define a categorizing function for total_income

def income_category(val):
    if val > 0 and val <= 17202:
        return 'Low income'
    elif val <= 22993: 
        return 'Below average income'
    elif val <= 31728:
        return 'Above average income'
    else:
        return 'High income'

In [21]:
# apply the above categorizing function on the total income column
credit_data['income_category'] = credit_data['total_income'].apply(income_category)

In [42]:
display(credit_data.head()) #visualize the new category column

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,have_kids
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620,House,High income,Yes
1,1,36,secondary education,1,married,0,F,employee,0,17932,Car,Below average income,Yes
2,0,33,secondary education,1,married,0,M,employee,0,23341,House,Above average income,No
3,3,32,secondary education,1,married,0,M,employee,0,42820,Education,High income,Yes
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378,Wedding,Above average income,No


In [23]:
#categorizing by loan purpose using lemmatization 
#learn about the different loan purposes
credit_data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [24]:
#creating the different categories for loan request
house = ['house', 'housing', 'estate', 'property']
car = ['car', 'cars', ]
education = ['education', 'educated', 'university']
wedding = ['wedding']

In [25]:
#defining the lemmatizing function and categoriez the purpose list we have into 4 groups as described above 


def lemmatizer_func(line):
    wordnet_lemma = WordNetLemmatizer()
    word_list = nltk.word_tokenize(line)
    word_list = [w.lower() for w in word_list]
    
    if any(word in word_list for word in house):
        return 'House'
    elif any(word in word_list for word in car):
        return 'Car'    
    elif any(word in word_list for word in education):
        return 'Education'  
    elif any(word in word_list for word in wedding):
        return 'Wedding'  
    else: 
        return 'Other'  


In [26]:
credit_data['purpose'] = credit_data['purpose'].apply(lemmatizer_func) # applying the above function on out data set

In [27]:
credit_data['purpose'].value_counts() #check the purpose category distribution 

House        10811
Car           4306
Education     4013
Wedding       2324
Name: purpose, dtype: int64

In [28]:
# categorizing people with or without kids
# defining a function for determining if the client has kids or not 
def have_kids(num_of_kids):
    if num_of_kids == 0:
        return 'No'
    else: 
        return 'Yes'

In [29]:
# creating a new column of "Have_kids"
credit_data['have_kids'] = credit_data['children'].apply(have_kids)

In [30]:
credit_data['have_kids'].value_counts() #check the have_kids category distribution 

No     14091
Yes     7363
Name: have_kids, dtype: int64

In [31]:
display(credit_data.head())

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category,have_kids
0,1,42,bachelor's degree,0,married,0,F,employee,0,40620,House,High income,Yes
1,1,36,secondary education,1,married,0,F,employee,0,17932,Car,Below average income,Yes
2,0,33,secondary education,1,married,0,M,employee,0,23341,House,Above average income,No
3,3,32,secondary education,1,married,0,M,employee,0,42820,Education,High income,Yes
4,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378,Wedding,Above average income,No


We categorized the "total income" into 4 groups - low income, below average, above average and high income and the loan purpose also into 4 groups - Car, house, education and wedding which fitted for all the data set (not value under "other").
approximately 60% of the people asking for a loan do not have kids. 

## Analysis

**- Is there a relation between having kids and repaying a loan on time?**

In order to check if there is a relation between having kids to repaying a loan on time we will create a pivot table with these values: "have_kids", "debt"

In [32]:
#creating a pivot table
kids_status = pd.pivot_table(credit_data, index = 'have_kids', columns = 'debt', values = 'children', aggfunc = 'count', margins = True)
kids_status

debt,0,1,All
have_kids,,,
No,13028,1063,14091
Yes,6685,678,7363
All,19713,1741,21454


In [43]:
#calculating the default rate
kids_status['default rate'] = (kids_status[1]/kids_status['All'])*100
display(kids_status)

debt,0,1,All,default rate
have_kids,,,,
No,13028,1063,14091,7.543822
Yes,6685,678,7363,9.208203
All,19713,1741,21454,8.115037


There is a relation between having kids to paying back a loan on time, yet it's not very significant. 
People who have kids have a higher default rate, meaning that comparing to people who don't have kids the don't repay loan on time. Yet, the diference is less than 2% between these groups. 
We can also take a look at the difference in you have a small family (1 kid) vs. a larger family. 

In [46]:
#creating a pivot table
num_kids_status = pd.pivot_table(credit_data, index = 'children', columns = 'debt', values = 'have_kids', aggfunc = 'count', margins = True)
display(num_kids_status)

debt,0,1,All
children,,,
0,13028.0,1063.0,14091
1,4410.0,445.0,4855
2,1926.0,202.0,2128
3,303.0,27.0,330
4,37.0,4.0,41
5,9.0,NaN,9
All,19713.0,1741.0,21454


In [47]:
#calculating the default rate
num_kids_status[1] = num_kids_status[1].fillna(0) #changing the Nan value for people with 5 kids and debt = 1 to 0 
num_kids_status['default_rate'] = num_kids_status[1]/num_kids_status['All']*100

display(num_kids_status)

debt,0,1,All,default_rate
children,,,,
0,13028.0,1063.0,14091,7.543822
1,4410.0,445.0,4855,9.165808
2,1926.0,202.0,2128,9.492481
3,303.0,27.0,330,8.181818
4,37.0,4.0,41,9.756098
5,9.0,0.0,9,0.000000
All,19713.0,1741.0,21454,8.115037


There no significant relation that we can obsorve between the number of children to the default rate.
We can see that all the people with 5 kids returned their loans on time, but we only have 9 people with 5 kids so it's irrelevant to make any conclusions on this regard. 

**- Is there a relation between marital status and repaying a loan on time?**

In [48]:
#creating a pivot table of "family status" and "debt"
family_status = pd.pivot_table(credit_data, index = 'family_status', columns = 'debt', values = 'family_status_id', aggfunc = 'count', margins = True)
display(family_status)

debt,0,1,All
family_status,,,
civil partnership,3763,388,4151
divorced,1110,85,1195
married,11408,931,12339
unmarried,2536,274,2810
widow / widower,896,63,959
All,19713,1741,21454


In [49]:
# calculating default rate
family_status['defualt_rate'] = family_status[1]/family_status['All']*100
display(family_status)

debt,0,1,All,defualt_rate
family_status,,,,
civil partnership,3763,388,4151,9.347145
divorced,1110,85,1195,7.112971
married,11408,931,12339,7.545182
unmarried,2536,274,2810,9.750890
widow / widower,896,63,959,6.569343
All,19713,1741,21454,8.115037


We can see widowers have the best defult rate, while unmarried have the worst. 
We can split to level of confidence: 
1. most trusted - widow / widower
2. medium trusted - married and divorced
3. least trusted - civil partnership and unmarried


**- Is there a relation between income level and repaying a loan on time?**

In [50]:
# creating a pivot table
income_status = pd.pivot_table(credit_data, index = 'income_category', columns = 'debt', values = 'income_type', aggfunc = 'count', margins = True)
display(income_status)

debt,0,1,All
income_category,,,
Above average income,4906,456,5362
Below average income,4878,485,5363
High income,4991,373,5364
Low income,4938,427,5365
All,19713,1741,21454


In [51]:
#calculating the default rate
income_status['defualt_rate'] = income_status[1]/income_status['All']*100
display(income_status)

debt,0,1,All,defualt_rate
income_category,,,,
Above average income,4906,456,5362,8.504289
Below average income,4878,485,5363,9.043446
High income,4991,373,5364,6.953766
Low income,4938,427,5365,7.958993
All,19713,1741,21454,8.115037


This data is somehow surpprising, we would have expected to get the be worst default rate for people with low income yet the worst were the people with around the average incomes (below and above average with very similar default rate). 
Yet, as we would have anticipated, people in the high income group had the best default rate, since money is probably not an issue in this group. 

**- How do different loan purposes affect on-time repayment of the loan?**

In [52]:
# creating a pivot table
purpose_status = pd.pivot_table(credit_data, index = 'purpose', columns = 'debt', values = 'income_type', aggfunc = 'count', margins = True)
display(purpose_status)

debt,0,1,All
purpose,,,
Car,3903,403,4306
Education,3643,370,4013
House,10029,782,10811
Wedding,2138,186,2324
All,19713,1741,21454


In [53]:
#calculating the default rate
purpose_status['defualt_rate'] = purpose_status[1]/purpose_status['All']*100
display(purpose_status)

debt,0,1,All,defualt_rate
purpose,,,,
Car,3903,403,4306,9.359034
Education,3643,370,4013,9.220035
House,10029,782,10811,7.233373
Wedding,2138,186,2324,8.003442
All,19713,1741,21454,8.115037


We can see that Car and education purposes have a higher default rate, 
while wedding have a lower defualt rate and house has the lowest. 
This could be due to a higher intrest rate for houase loans, people prefer to pay them on time. 
Also, education loans are dependent on whether you completed your studies and found a job that can pay back the loan quickly. 


## Conclusions

After examining the data I received, performing some preprocessing methods to clean it, such as replacing missing values and unreasonable values, categorizing the data into the groups of interest - kids status, income level and loan purpose I created several pivot tables to answer the intresting questions requested. 

From the information gathered above we can conclude from the dataset given that the poplution that is more likely to return loans on time will be composed of the following:
1. People with no kids.
2. If they have kids it dosn't really matter how many. 
3. People who are widows / widowers, married or divorced are more trustworthy.
4. people with high income level (above 31,703 was tested in this dataset).
5. People who ask for a loan for the purpose of buying a house or planning a wedding.